In [40]:
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import collections

In [41]:
# read names that have provided survey eye color data
columns = ['name', 'timestamp', 'id', 'blood_type', 'height', 'weight', 'hw_comments', 'left', 'right', 'left_desc', 'right_desc', 'eye_comments', 'hair', 'hair_desc', 'hair_comments', 'misc', 'handedness']

# pgp eye color data from survey
surveyData = pd.read_csv("./eye_color_data/PGP-Survey.csv", names=columns, na_values=['nan', '', 'NaN'])

# names of the pgp participants
surveyNames = np.asarray(surveyData['name'].values.tolist())

# load numpy array of tiled PGP data 
pgp = preprocessing.scale(np.load("hiq-pgp").astype('double'))

In [42]:
# load numpy array of names and keep only the huID
pgpNames = np.load("names")
pgpNames = map(lambda name: name[:8], pgpNames)

# simple lambda function to return if the input is a string
isstr = lambda val: isinstance(val, str)

In [43]:
eye_color = collections.namedtuple("EyeColor", ['left', 'right'])

# lookup a name in the survey data and return a tuple of the eye colors
def getAllData(name, surveyData):
    for index, row in surveyData.iterrows():
        if row['name'] == name:
            return eye_color(row['left'], row['right'])
        
# lookup a name in the survey data and return a tuple of the eye colors, excluding hazel
def getDataNoHazel(name, surveyData):
    for index, row in surveyData.iterrows():
        if row['name'] == name:
            if isstr(row['left_desc']) and isstr(row['right_desc']):
                if 'azel' in row['left_desc'] or 'azel' in row['right_desc']:
                    return None
            return eye_color(row['left'], row['right'])

In [44]:
# list of tuples for index and name with eye color data (idx, name)
nameEyeMap = []
namePair = collections.namedtuple("NamePair", ['index', 'name'])

# dictionary of left and right eye colors with respective name, i.e., {"huID": 12}
leftEyeMap = {}
rightEyeMap = {}

existingNames = []

# loop through pgpNames and add eye color to maps, making sure not to add the same name twice
for i, name in enumerate(pgpNames):
    if name in surveyNames and name not in existingNames:
        existingNames.append(name)
        eyeData = getDataNoHazel(name, surveyData)
        if eyeData == None:
            pass
        elif isstr(eyeData.left) and isstr(eyeData.right):
            nameEyeMap.append(namePair(i, name))
            leftEyeMap[name] = eyeData.left
            rightEyeMap[name] = eyeData.right

# create lists containing the known eye color names and the unknown eye colors.
nameIndices, correspondingNames = [], []
for pair in nameEyeMap:
    nameIndices.append(pair.index)
    correspondingNames.append(pair.name)
knownData = pgp[nameIndices]
unknownData = np.delete(pgp, nameIndices, axis=0)

In [ ]:
# convert dictionaries to lists 
leftEyeNameList = []
rightEyeNameList = []
# nametuple looks like (index, name)
for _, name in nameEyeMap:
    if isstr(leftEyeMap[name]):
        leftEyeNameList.append(leftEyeMap[name])
    if isstr(rightEyeMap[name]):
        rightEyeNameList.append(rightEyeMap[name])

blueOrNot = lambda color: 0 if int(color) > 13 else 1
leftEyeNameList = map(blueOrNot, leftEyeNameList)

In [ ]:
# create histogram of blue/not blue. TODO: make labels for graph

plt.hist(leftEyeNameList)
plt.ylabel("Number of Participants")
plt.xlabel("Eye Color")
plt.show()

In [ ]:
def getDesc(name, surveyData):
    for index, row in surveyData.iterrows():
        if row['name'] == name:
            return (row['left_desc'], row['right_desc'])

In [53]:
# 10-fold cross validation
svc_test = LinearSVC(penalty='l1', class_weight='balanced', 
                     C=.06, dual=False, verbose=1, max_iter=1000)
cv_scores = cross_val_score(svc_test, knownData, leftEyeNameList, cv=10, scoring='accuracy')
print
print cv_scores.mean()
print cv_scores.std()

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]
0.945714285714
0.0842251598456


In [56]:
# leave one out validation
svc_test = LinearSVC(penalty='l1', class_weight='balanced', 
                     C=.06, dual=False, verbose=1, max_iter=1000)
from sklearn.model_selection import LeaveOneOut

leftOut = LeaveOneOut()
leftEyeNameList = np.asarray(leftEyeNameList)
scores = []
counter = 0
predicted = []
tested = []
for train_index, test_index in leftOut.split(knownData):
    X_train, X_test = knownData[train_index], knownData[test_index]
    y_train, y_test = leftEyeNameList[train_index], leftEyeNameList[test_index]
    svc_test.fit(X_train, y_train)
    y_pred = svc_test.predict(X_test)
    
    scores.append(accuracy_score(y_test, y_pred))
    predicted.append(y_pred)
    tested.append(y_test)
    
    counter += 1
    print "Iteration: " + str(counter) + ", correct: " + str(bool(y_test == y_pred))
    if not bool(y_test == y_pred):
        print "Incorrect: ", test_index

[LibLinear]Iteration: 1, correct: True
[LibLinear]Iteration: 2, correct: True
[LibLinear]Iteration: 3, correct: True
[LibLinear]Iteration: 4, correct: True
[LibLinear]Iteration: 5, correct: True
[LibLinear]Iteration: 6, correct: False
Incorrect:  [5]
[LibLinear]Iteration: 7, correct: True
[LibLinear]Iteration: 8, correct: True
[LibLinear]Iteration: 9, correct: True
[LibLinear]Iteration: 10, correct: True
[LibLinear]Iteration: 11, correct: True
[LibLinear]Iteration: 12, correct: True
[LibLinear]Iteration: 13, correct: True
[LibLinear]Iteration: 14, correct: True
[LibLinear]Iteration: 15, correct: True
[LibLinear]Iteration: 16, correct: True
[LibLinear]Iteration: 17, correct: True
[LibLinear]Iteration: 18, correct: True
[LibLinear]Iteration: 19, correct: True
[LibLinear]Iteration: 20, correct: True
[LibLinear]Iteration: 21, correct: True
[LibLinear]Iteration: 22, correct: True
[LibLinear]Iteration: 23, correct: True
[LibLinear]Iteration: 24, correct: True
[LibLinear]Iteration: 25, correc

In [55]:
# print results of leave one out validation
print np.mean(np.asarray(scores))
import collections
print collections.Counter(scores)

0.950819672131
Counter({1.0: 58, 0.0: 3})


In [65]:
incorrGuesses = np.asarray([5, 51, 54])
correspondingNames = np.asarray(correspondingNames)

In [66]:
for item in correspondingNames[incorrGuesses]:
    print item

hu0D1FA1
huDBD591
huE9B698


In [ ]:
# determining of the eye color can be solely determined by the tile at position 1792420. 
# While it is more accurate than the classifier 
guesses = []
for val in range(len(knownData)):
    one_hot = ['not blue', 'blue']
    tile = knownData[val][1792420]
    if tile < 0: # not blue
        guesses.append(0)
    else: # blue
        guesses.append(1)
print accuracy_score(leftEyeNameList, guesses)

In [ ]:
incorrTiles = []
for val in range(len(knownData)):
    one_hot = ['not blue', 'blue']
    tile = knownData[val][1792420]
    prediction = one_hot[svc_test.predict([knownData[val]])[0]]
    if tile < 0:
        if prediction == "not blue": 
            pass
        else: 
            print val
            incorrTiles.append(val)
    else:
        if prediction == "blue":
            pass
        else:
            print val
            incorrTiles.append(val)

In [ ]:
knownDataFiltered = np.delete(knownData, incorrTiles, axis=0)
leftEyeNameListFiltered = np.delete(leftEyeNameList, incorrTiles, axis=0)
cv_scores = cross_val_score(svc_test, knownDataFiltered, leftEyeNameListFiltered, cv=10, scoring='accuracy')
print
print cv_scores.mean()
print cv_scores.std()

In [ ]:
svc_test.fit(knownDataFiltered, leftEyeNameListFiltered)

# retrieve all the nonzero coefficients and zip them with their respective indices
nonzeroes = np.nonzero(svc_test.coef_[0])[0]
coefs = zip(nonzeroes, svc_test.coef_[0][nonzeroes])

# sort the coefficients by their value, instead of index
coefs.sort(key = lambda x: x[1], reverse=True)

for coef in coefs:
    print coef

In [ ]:
corrNames = np.asarray(corrNames)

print corrNames[62]

In [ ]:
svc_test.fit(knownDataFiltered, leftEyeNameListFiltered)

In [ ]:
print svc_test.predict(knownData[incorrTiles])
print leftEyeNameList[incorrTiles]

In [ ]:
# create confusion matrix - code taken from Sarah's blood type prediction
cnf_matrix = confusion_matrix(predicted, tested)
plt.imshow(cnf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.ylabel('True label')
plt.xlabel('Predicted label')

classes = ['Blue','Not Blue']

plt.grid('off')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

for i in range(cnf_matrix.shape[0]):
    for j in range(cnf_matrix.shape[1]):
        plt.text(j, i, cnf_matrix[i, j],
                 horizontalalignment="center",
                 color= "orangered")

plt.gcf().subplots_adjust(left=0.25, bottom = 0.35)

plt.savefig('Blue_Confusion.png',format='png',dpi=300)